In [1]:
# ================================
# 1. AUTHENTICATION & ENVIRONMENT
# ================================
# Detects the runtime environment (Kaggle vs local) and configures
# the HuggingFace token accordingly.
# - Kaggle  : reads HF_TOKEN from Kaggle Secrets
# - Local   : uses cached token or triggers notebook_login()

import os

def is_kaggle():
    """Check if the notebook is running inside a Kaggle kernel."""
    return os.path.exists("/kaggle/working")

if is_kaggle():
    from kaggle_secrets import UserSecretsClient

    # Redirect HF cache to the Kaggle working directory (persistent storage)
    os.environ["HF_HOME"] = "/kaggle/working/hf_cache"

    hf_token = UserSecretsClient().get_secret("HF_TOKEN")
    os.environ["HF_TOKEN"] = hf_token

    print("✅ Running on Kaggle — HF token loaded from Kaggle Secrets")

else:
    from huggingface_hub import get_token

    if get_token() is None:
        # No cached token found — prompt the user to log in
        from huggingface_hub import notebook_login
        notebook_login()
    else:
        print("✅ Running locally — HF token found in cache")

✅ Running on Kaggle — HF token loaded from Kaggle Secrets


In [2]:
pip install --upgrade --quiet accelerate bitsandbytes transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 32.0 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
# ================================
# LOGGING CONFIGURATION
# ================================
# Sets up a centralized logger used across all cells.
# - INFO  : normal flow (profile saved, agent called, result returned)
# - WARNING : non-critical issues (empty input, missing profile)
# - ERROR : failures (JSON parse error, model retry exhausted)

import logging

# Create a named logger so we don't pollute the root logger
logger = logging.getLogger("prenatal_care")
logger.setLevel(logging.DEBUG)

# Avoid duplicate handlers when the cell is re-run
if not logger.handlers:
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    formatter = logging.Formatter(
        "[%(asctime)s] %(levelname)s | %(message)s",
        datefmt="%H:%M:%S"
    )
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)

logger.info("Logger initialised — level: %s", logger.level)

[12:22:07] INFO | Logger initialised — level: 10


# Load Model Gemini MedGemma 4B-it 

In [5]:
# ================================
# 2. LOAD MODEL — MedGemma 4B-IT
# ================================
# Loads google/medgemma-4b-it with 4-bit quantization via BitsAndBytes.
# The model supports both text-only and image+text inputs (image-text-to-text).
# - dtype        : bfloat16 for reduced memory on GPU
# - device_map   : "auto" lets accelerate place layers across available devices
# - do_sample    : False → greedy decoding for deterministic outputs

from transformers import pipeline, BitsAndBytesConfig
import torch

model_variant = "4b-it"
model_id = f"google/medgemma-{model_variant}"

print(f"⏳ Loading model: {model_id} (4-bit quantized) ...")

model_kwargs = dict(
    dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=BitsAndBytesConfig(load_in_4bit=True)
)

pipe = pipeline(
    "image-text-to-text",
    model=model_id,
    model_kwargs=model_kwargs,
)

# Use greedy decoding for consistent, reproducible responses
pipe.model.generation_config.do_sample = False

print(f"✅ Model loaded successfully on device: {pipe.device}")

⏳ Loading model: google/medgemma-4b-it (4-bit quantized) ...


Loading weights:   0%|          | 0/883 [00:00<?, ?it/s]

✅ Model loaded successfully on device: cuda:0


### Extract JSON 

In [6]:
# ================================
# 3. JSON EXTRACTION UTILITY
# ================================
# The model sometimes returns JSON wrapped in markdown fences or extra text.
# This function robustly extracts the JSON object from raw model output.
#
# Strategy (two-pass):
#   1. Greedy match: find the outermost { ... } and parse it.
#   2. Truncation recovery: if the JSON is cut off (max_new_tokens limit),
#      close unclosed braces/brackets and retry.

import json
import re

def extract_json(text):
    """Extract a JSON object from model output, with truncation recovery."""

    # --- Pass 1: direct parse of the largest {...} block ---
    try:
        match = re.search(r"\{.*\}", text, re.DOTALL)
        if match:
            parsed = json.loads(match.group())
            logger.debug("extract_json: direct parse succeeded")
            return parsed
    except json.JSONDecodeError as e:
        logger.debug("extract_json: direct parse failed — %s", e)

    # --- Pass 2: attempt to repair truncated JSON ---
    try:
        match = re.search(r"\{.*", text, re.DOTALL)
        if match:
            fragment = match.group().rstrip()
            # Strip trailing comma that would make JSON invalid
            fragment = re.sub(r",\s*$", "", fragment)
            # Count unclosed delimiters
            open_braces = fragment.count("{") - fragment.count("}")
            open_brackets = fragment.count("[") - fragment.count("]")
            # Close a dangling string literal
            if fragment.count('"') % 2 != 0:
                fragment += '"'
            # Append missing closing delimiters
            fragment += "]" * max(open_brackets, 0)
            fragment += "}" * max(open_braces, 0)

            parsed = json.loads(fragment)
            logger.warning("extract_json: used truncation recovery (added %d '}', %d ']')",
                           max(open_braces, 0), max(open_brackets, 0))
            return parsed
    except (json.JSONDecodeError, Exception) as e:
        logger.error("extract_json: truncation recovery also failed — %s", e)

    logger.error("extract_json: could not extract JSON from model output")
    return None

### Model Calling 

In [7]:
# ================================
# 4. MODEL CALLING WRAPPER
# ================================
# Sends a list of chat messages to the MedGemma pipeline, extracts JSON
# from the response, and retries up to `retries` times if parsing fails.
#
# Returns:
#   dict — parsed JSON on success, or {"error": "..."} after all retries.

def call_model(messages, retries=3):
    """Call MedGemma and return the parsed JSON response."""
    for attempt in range(1, retries + 1):
        logger.info("call_model: attempt %d/%d", attempt, retries)

        output = pipe(
            messages,
            max_new_tokens=1024   # raised from 512 to avoid truncation
        )

        # The pipeline returns a list; grab the assistant's last message
        text = output[0]["generated_text"][-1]["content"]
        logger.debug("call_model: raw output length = %d chars", len(text))

        parsed = extract_json(text)
        if parsed:
            logger.info("call_model: success on attempt %d", attempt)
            return parsed

        logger.warning("call_model: JSON extraction failed on attempt %d", attempt)

    logger.error("call_model: all %d retries exhausted — returning error dict", retries)
    return {"error": "Model failed to return valid JSON after retries"}

### Input user profile for personalized recommendations

In [8]:
# ================================
# 5. USER PROFILE INPUT FORM
# ================================
# Collects the patient's personal & medical details once via terminal prompts.
# The returned dict is shared across ALL agent functions so every
# recommendation is personalised to the individual.
#
# Fields captured:
#   name, age, weight, height, pregnancy_week, pregnancy_type,
#   num_fetuses, medical_history, allergies, blood_group, pregnancy_history

def collect_user_profile():
    """Interactive terminal form that returns a user-profile dict."""
    print("=" * 50)
    print("  PRENATAL CARE — USER PROFILE SETUP")
    print("=" * 50)

    # --- Basic demographics ---
    name = input("Enter your name: ").strip()
    age = int(input("Enter your age: ").strip())
    weight = float(input("Enter your weight (kg): ").strip())
    height = float(input("Enter your height (cm): ").strip())
    week = int(input("Current pregnancy week (1-42): ").strip())

    # --- Pregnancy type (single / multiple) ---
    pregnancy_type = input("Pregnancy type — single or multiple? (single/multiple): ").strip().lower()
    while pregnancy_type not in ("single", "multiple"):
        pregnancy_type = input("Please enter 'single' or 'multiple': ").strip().lower()

    num_fetuses = 1
    if pregnancy_type == "multiple":
        num_fetuses = int(input("How many fetuses? (2/3/...): ").strip())

    # --- Medical history (comma-separated or 'none') ---
    history_input = input("Any medical history? (comma-separated, e.g. diabetes,hypertension or 'none'): ").strip()
    medical_history = [] if history_input.lower() == "none" else [h.strip() for h in history_input.split(",")]

    # --- Allergies ---
    allergies_input = input("Any known allergies? (comma-separated or 'none'): ").strip()
    allergies = [] if allergies_input.lower() == "none" else [a.strip() for a in allergies_input.split(",")]

    # --- Blood group & pregnancy history ---
    blood_group = input("Blood group (e.g. A+, B-, O+, or 'unknown'): ").strip()

    is_first = input("Is this your First Pregnancy? (yes/no): ").strip().lower() == "yes"
    pregnancy_history = (
        "First Pregnancy (has no previous pregnancy experience)"
        if is_first
        else "Not First Pregnancy (has previous pregnancy experience)"
    )

    profile = {
        "name": name,
        "age": age,
        "weight_kg": weight,
        "height_cm": height,
        "pregnancy_week": week,
        "pregnancy_type": pregnancy_type,
        "num_fetuses": num_fetuses,
        "medical_history": medical_history,
        "allergies": allergies,
        "blood_group": blood_group,
        "pregnancy_history": pregnancy_history,
    }

    # Log and display the saved profile
    logger.info("User profile created for '%s' (age %d, week %d, %s pregnancy)",
                name, age, week, pregnancy_type)

    print("\n" + "=" * 50)
    print("  ✅ PROFILE SAVED SUCCESSFULLY")
    print("=" * 50)
    for k, v in profile.items():
        print(f"  {k}: {v}")
    print("=" * 50)

    return profile

# --- Run the form — the returned profile is used by all subsequent agents ---
# user_profile = collect_user_profile() # uncomment if running terminal based chat


  PRENATAL CARE — USER PROFILE SETUP


Enter your name:  hello
Enter your age:  45
Enter your weight (kg):  124
Enter your height (cm):  157
Current pregnancy week (1-42):  15
Pregnancy type — single or multiple? (single/multiple):  single
Any medical history? (comma-separated, e.g. diabetes,hypertension or 'none'):  none
Any known allergies? (comma-separated or 'none'):  none
Blood group (e.g. A+, B-, O+, or 'unknown'):  b+
Is this your First Pregnancy? (yes/no):  yes


[12:24:18] INFO | User profile created for 'hello' (age 45, week 15, single pregnancy)



  ✅ PROFILE SAVED SUCCESSFULLY
  name: hello
  age: 45
  weight_kg: 124.0
  height_cm: 157.0
  pregnancy_week: 15
  pregnancy_type: single
  num_fetuses: 1
  medical_history: []
  allergies: []
  blood_group: b+
  pregnancy_history: First Pregnancy (has no previous pregnancy experience)


# Agent - 1 Symptom Analyser

In [9]:
# ================================
# AGENT 1 — SYMPTOM ANALYZER
# ================================
# Takes a free-text symptom description and the user profile.
# Returns a JSON risk assessment: risk_level, reasoning, advice,
# personalised note, and whether a doctor visit is required.
#
# Special behaviour:
#   - For MULTIPLE pregnancies → more cautious risk classification
#   - If risk_level == "Urgent" → advice is overridden to "Consult doctor immediately"

def analyze_symptoms(user_input, user_profile):
    """Analyse reported symptoms in the context of the patient's profile."""

    logger.info("analyze_symptoms: called — input='%s'", user_input[:80])

    # Build a one-line profile summary injected into the system prompt
    profile_summary = (
        f"Patient: {user_profile['name']}, Age: {user_profile['age']}, "
        f"Week: {user_profile['pregnancy_week']}, "
        f"Pregnancy type: {user_profile['pregnancy_type']} "
        f"({user_profile['num_fetuses']} fetus(es)), "
        f"Medical history: {', '.join(user_profile['medical_history']) or 'None'}, "
        f"Allergies: {', '.join(user_profile['allergies']) or 'None'}, "
        f"Pregnancy history: {user_profile.get('pregnancy_history')}"
    )

    system_prompt = f"""
You are a prenatal medical AI assistant.

Patient profile:
{profile_summary}

Analyze the reported symptoms in context of this specific patient's profile.

Rules:
- Classify risk: Normal / Caution / Urgent
- Consider the patient's age, pregnancy week, pregnancy type (single vs multiple), and medical history
- For MULTIPLE pregnancies, be MORE cautious and flag risks that may not apply to single pregnancies
- Be medically cautious
- No assumptions
- ALWAYS return JSON ONLY

Output format:
{{
  "risk_level": "...",
  "reasoning": "...",
  "advice": "...",
  "personalized_note": "...",
  "doctor_required": true/false
}}
"""

    messages = [
        {"role": "system", "content": [{"type": "text", "text": system_prompt}]},
        {"role": "user", "content": [{"type": "text", "text": user_input}]}
    ]

    result = call_model(messages)

    # Safety override — always direct urgent cases to a doctor
    if result.get("risk_level") == "Urgent":
        result["advice"] = "Consult doctor immediately"
        logger.warning("analyze_symptoms: URGENT risk detected — advice overridden")

    logger.info("analyze_symptoms: risk_level='%s'", result.get("risk_level", "unknown"))
    return result

# Agent - 2 Report Analyser

In [10]:
# ================================
# AGENT 2 — REPORT EXPLAINER
# ================================
# Accepts either a text report, an image (e.g. scanned lab sheet), or both.
# Dynamically identifies medical values present and explains each finding
# in simple language, personalised to the patient's profile.
#
# Returns JSON with: findings[], summary, personalized_note

def explain_report(user_profile, text=None, image=None):
    """Explain a medical report (text and/or image) in patient context."""

    logger.info("explain_report: called — text=%s, image=%s",
                bool(text), bool(image))

    profile_summary = (
        f"Patient: {user_profile['name']}, Age: {user_profile['age']}, "
        f"Week: {user_profile['pregnancy_week']}, "
        f"Pregnancy type: {user_profile['pregnancy_type']} "
        f"({user_profile['num_fetuses']} fetus(es)), "
        f"Medical history: {', '.join(user_profile['medical_history']) or 'None'}, "
        f"Pregnancy history: {user_profile['pregnancy_history']}"
    )

    system_prompt = f"""
You are a prenatal report analysis AI.

Patient profile:
{profile_summary}

Extract medical insights dynamically and personalize findings for this patient.

Rules:
- DO NOT assume fixed fields
- Identify any medical values present
- Explain in simple language
- For MULTIPLE pregnancies, note findings specific to multiples
- Consider patient's medical history when assessing risk
- Return JSON ONLY

Format:
{{
  "findings": [
    {{
      "name": "...",
      "value": "...",
      "meaning": "...",
      "risk": "Normal/Caution/Urgent"
    }}
  ],
  "summary": "...",
  "personalized_note": "..."
}}
"""

    # Build user content — may include text, image, or both
    content = []
    if text:
        content.append({"type": "text", "text": text})
    if image:
        content.append({"type": "image", "image": image})

    if not content:
        logger.warning("explain_report: neither text nor image provided")
        return {"error": "No report content provided (text or image required)"}

    messages = [
        {"role": "system", "content": [{"type": "text", "text": system_prompt}]},
        {"role": "user", "content": content}
    ]

    result = call_model(messages)
    num_findings = len(result.get("findings", [])) if isinstance(result, dict) else 0
    logger.info("explain_report: returned %d findings", num_findings)
    return result

# Agent - 3 Week Tracker

In [11]:
# ================================
# AGENT 3 — WEEK TRACKER
# ================================
# Provides week-specific pregnancy guidance: trimester info, expected
# symptoms, tips, and doctor reminders — all personalised to the patient.
#
# Extra logic:
#   - Multiple pregnancies → extra monitoring guidance
#   - First-time mothers   → reassurance & beginner tips

def week_tracker(user_profile):
    """Return personalised guidance for the current pregnancy week."""

    week = user_profile["pregnancy_week"]
    logger.info("week_tracker: called for week %d", week)

    profile_summary = (
        f"Patient: {user_profile['name']}, Age: {user_profile['age']}, "
        f"Pregnancy type: {user_profile['pregnancy_type']} "
        f"({user_profile['num_fetuses']} fetus(es)), "
        f"Medical history: {', '.join(user_profile['medical_history']) or 'None'}, "
        f"Allergies: {', '.join(user_profile['allergies']) or 'None'}, "
        f"Pregnancy history: {user_profile['pregnancy_history']}"
    )

    # Output format kept as a plain string (not f-string) to avoid brace issues
    output_format = """
{
  "trimester": "...",
  "tips": ["..."],
  "symptoms": ["..."],
  "doctor_reminders": ["..."],
  "personalized_note": "..."
}
"""

    system_prompt = f"""
You are a pregnancy care assistant. Provide information for pregnancy week {week}.

Patient profile:
{profile_summary}

Rules:
- Personalize tips based on the patient's age, medical history, pregnancy type, and allergies
- For MULTIPLE pregnancies, include specific guidance (extra rest, more frequent monitoring, etc.)
- For first-time mothers, include extra reassurance and beginner tips
- ALWAYS return your response as a STRICT JSON object. No other text or conversation is allowed.

Output format:
""" + output_format

    messages = [
        {"role": "system", "content": [{"type": "text", "text": system_prompt}]},
        {"role": "user", "content": [{"type": "text", "text": f"Provide personalized precautionary steps for pregnancy week {week}."}]}
    ]

    result = call_model(messages)
    logger.info("week_tracker: trimester='%s'", result.get("trimester", "unknown"))
    return result

# Agent - 4 Daily Care Planner

In [12]:
# ================================
# AGENT 4 — DAILY CARE PLANNER
# ================================
# Generates a personalised daily care plan covering diet, hydration,
# exercise, rest, and special notes.
#
# Adapts recommendations based on:
#   - Allergies & medical conditions (e.g. GDM, hypertension)
#   - Pregnancy type (lighter exercise / more rest for multiples)
#   - BMI (weight + height) and current pregnancy week

def daily_care(user_profile):
    """Generate a personalised prenatal daily care plan."""

    logger.info("daily_care: called for '%s', week %d",
                user_profile['name'], user_profile['pregnancy_week'])

    profile_summary = (
        f"Patient: {user_profile['name']}, Age: {user_profile['age']}, "
        f"Weight: {user_profile['weight_kg']}kg, Height: {user_profile['height_cm']}cm, "
        f"Week: {user_profile['pregnancy_week']}, "
        f"Pregnancy type: {user_profile['pregnancy_type']} "
        f"({user_profile['num_fetuses']} fetus(es)), "
        f"Medical history: {', '.join(user_profile['medical_history']) or 'None'}, "
        f"Allergies: {', '.join(user_profile['allergies']) or 'None'}, "
        f"Blood group: {user_profile['blood_group']}, "
        f"Pregnancy history: {user_profile['pregnancy_history']}"
    )

    # Output format as plain string to avoid f-string brace conflicts
    output_format = """
{
  "diet": ["..."],
  "foods_to_avoid": ["..."],
  "hydration": "...",
  "exercise": ["..."],
  "rest_recommendation": "...",
  "special_notes": "..."
}
"""

    system_prompt = f"""
You are a pregnancy care assistant. Generate a personalized prenatal daily care plan.

Patient profile:
{profile_summary}

Rules:
- Adapt diet based on allergies, medical history and nutritional needs for the pregnancy week
- Adapt exercise intensity based on age, weight, pregnancy type and week
- For MULTIPLE pregnancies, recommend lighter exercise, more rest, and higher caloric intake
- For patients with conditions like diabetes or hypertension, tailor diet and exercise accordingly
- ALWAYS return your response as a STRICT JSON object. No other text or conversation is allowed.
- Keep each field value concise.

Output format:
""" + output_format

    messages = [
        {"role": "system", "content": [{"type": "text", "text": system_prompt}]},
        {"role": "user", "content": [{"type": "text", "text": "Generate a personalized daily care plan."}]}
    ]

    result = call_model(messages)
    logger.info("daily_care: plan generated — %d diet items, %d exercises",
                len(result.get("diet", [])), len(result.get("exercise", [])))
    return result

In [13]:
# ================================
# UTILITY — EXTRACT MODEL RESPONSE
# ================================
# A flexible response normaliser that handles multiple output shapes:
#   Case 1: HuggingFace pipeline dict  → dig into generated_text
#   Case 2: Raw string                 → extract JSON from it
#   Case 3: Already-parsed dict        → return as-is
#   Case 4: List                       → use last element
#
# Used by the terminal-based test cells to display clean output.

import json
import re

def extract_model_response(data):
    """Normalise various model output shapes into a clean dict."""
    raw_output = None

    # Case 1: HuggingFace pipeline format
    if isinstance(data, dict) and "generated_text" in data:
        raw_output = data["generated_text"][-1]["content"]
        if isinstance(raw_output, list):
            raw_output = raw_output[0].get("text", "")
        logger.debug("extract_model_response: HF pipeline format detected")

    # Case 2: Plain string
    elif isinstance(data, str):
        raw_output = data

    # Case 3: Already a clean dict (most common from call_model)
    elif isinstance(data, dict):
        return data

    # Case 4: List response
    elif isinstance(data, list):
        raw_output = str(data[-1])

    else:
        logger.warning("extract_model_response: unrecognised data type %s", type(data))
        return None

    # Try to extract a JSON block from the string
    match = re.search(r"\{.*\}", raw_output, re.DOTALL)
    if not match:
        logger.warning("extract_model_response: no JSON found in output")
        return None

    try:
        return json.loads(match.group(0))
    except json.JSONDecodeError:
        logger.error("extract_model_response: JSON decode failed")
        return None

In [14]:
# Verify the user profile collected earlier
print(json.dumps(user_profile, indent=2))

{
  "name": "hello",
  "age": 45,
  "weight_kg": 124.0,
  "height_cm": 157.0,
  "pregnancy_week": 15,
  "pregnancy_type": "single",
  "num_fetuses": 1,
  "medical_history": [],
  "allergies": [],
  "blood_group": "b+",
  "pregnancy_history": "First Pregnancy (has no previous pregnancy experience)"
}


In [48]:
# --- Test: Symptom Analyzer ---
# Send a sample symptom to Agent 1 and display the parsed result.
model_response = analyze_symptoms("I have my foots swollen and back pain is it normal?", user_profile)
print(json.dumps(extract_model_response(model_response), indent=2))

[13:10:10] INFO | analyze_symptoms: called — input='I have my foots swollen and back pain is it normal?'
[13:10:10] INFO | call_model: attempt 1/3
Both `max_new_tokens` (=1024) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
[13:10:45] INFO | call_model: success on attempt 1
[13:10:45] INFO | analyze_symptoms: risk_level='Caution'


{
  "risk_level": "Caution",
  "reasoning": "Foot swelling (edema) and back pain can be common in pregnancy, especially in the second trimester. However, it's important to consider the severity and context. Swelling can be a sign of preeclampsia, a serious pregnancy complication. Back pain can be due to hormonal changes, weight gain, or musculoskeletal strain. Since this is the patient's first pregnancy, it's crucial to rule out any underlying conditions.",
  "advice": "Monitor your foot swelling and back pain. If the swelling is severe, especially if it's only in one leg, or if it's accompanied by headaches, vision changes, or upper abdominal pain, seek immediate medical attention. If the back pain is severe and doesn't improve with rest and over-the-counter pain relievers, consult your doctor. Consider keeping a log of your symptoms, including when they occur, what makes them better or worse, and any other associated symptoms.",
  "personalized_note": "As this is your first pregnancy

In [49]:
# --- Test: Week Tracker ---
# Fetch personalised tips for the user's current pregnancy week.
model_response = week_tracker(user_profile)
print(json.dumps(extract_model_response(model_response), indent=2))

[13:11:00] INFO | week_tracker: called for week 15
[13:11:00] INFO | call_model: attempt 1/3
Both `max_new_tokens` (=1024) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
[13:12:14] INFO | call_model: success on attempt 1
[13:12:14] INFO | week_tracker: trimester='Second Trimester'


{
  "trimester": "Second Trimester",
  "tips": [
    "Congratulations on reaching week 15! You're well into your second trimester, which is often considered the most comfortable phase of pregnancy.",
    "Continue to eat a healthy, balanced diet rich in fruits, vegetables, lean protein, and whole grains. Focus on getting enough folic acid, iron, and calcium.",
    "Stay hydrated by drinking plenty of water throughout the day. Aim for at least 8 glasses.",
    "Continue to rest and avoid strenuous activities. Listen to your body and take breaks when needed.",
    "Consider prenatal yoga or swimming to help ease discomfort and improve your overall well-being.",
    "If you're experiencing morning sickness, try eating small, frequent meals and avoiding strong odors. Ginger tea or ginger candies may also help.",
    "If you're experiencing heartburn, avoid spicy foods, fatty foods, and lying down immediately after eating. Elevate the head of your bed.",
    "Consider using a maternity supp

In [50]:
# --- Test: Daily Care Planner ---
# Generate a full daily care plan (diet, exercise, rest) for the user.
model_response = daily_care(user_profile)
print(json.dumps(extract_model_response(model_response), indent=2))

[13:12:14] INFO | daily_care: called for 'user', week 15
[13:12:14] INFO | call_model: attempt 1/3
Both `max_new_tokens` (=1024) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
[13:12:51] INFO | call_model: success on attempt 1
[13:12:51] INFO | daily_care: plan generated — 7 diet items, 4 exercises


{
  "diet": [
    "Focus on nutrient-rich foods to support fetal development.",
    "Increase protein intake for tissue growth.",
    "Consume iron-rich foods to prevent anemia.",
    "Include calcium for bone development.",
    "Ensure adequate folate intake for neural tube development.",
    "Small, frequent meals are recommended.",
    "Consider prenatal vitamins as a supplement to a healthy diet."
  ],
  "foods_to_avoid": [
    "Raw or undercooked meat, poultry, and seafood.",
    "Unpasteurized dairy products.",
    "Certain types of fish (high in mercury).",
    "Alcohol.",
    "Excessive caffeine.",
    "Processed foods with artificial sweeteners."
  ],
  "hydration": "Drink at least 8-10 glasses of water daily.",
  "exercise": [
    "Gentle walking for 20-30 minutes.",
    "Swimming (if comfortable and safe).",
    "Prenatal yoga (with instructor approval).",
    "Avoid strenuous activities and high-impact exercises."
  ],
  "rest_recommendation": "Get adequate sleep (7-8 hours

In [51]:
# --- Test: Report Explainer ---
# Feed a sample lab report to Agent 2 and display the findings.
sample_report = """
Hemoglobin: 9.5 g/dL
Blood Pressure: 140/90 mmHg
Fasting Glucose: 110 mg/dL
"""

print("REPORT EXPLAINER:")
result = explain_report(user_profile, text=sample_report)
print(json.dumps(result, indent=2))

[13:12:51] INFO | explain_report: called — text=True, image=False
[13:12:51] INFO | call_model: attempt 1/3
Both `max_new_tokens` (=1024) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


REPORT EXPLAINER:


[13:13:38] INFO | call_model: success on attempt 1
[13:13:38] INFO | explain_report: returned 3 findings


{
  "findings": [
    {
      "name": "Hemoglobin",
      "value": "9.5 g/dL",
      "meaning": "Hemoglobin is a protein in red blood cells that carries oxygen. A level of 9.5 g/dL is slightly low.",
      "risk": "Caution"
    },
    {
      "name": "Blood Pressure",
      "value": "140/90 mmHg",
      "meaning": "Blood pressure is the force of your blood against your artery walls. A reading of 140/90 mmHg is considered elevated and may indicate hypertension.",
      "risk": "Caution"
    },
    {
      "name": "Fasting Glucose",
      "value": "110 mg/dL",
      "meaning": "Fasting glucose is the amount of glucose in your blood after not eating for at least 8 hours. A level of 110 mg/dL is considered elevated and may indicate gestational diabetes.",
      "risk": "Caution"
    }
  ],
  "summary": "This patient is at 15 weeks pregnant and has slightly low hemoglobin, elevated blood pressure, and elevated fasting glucose. These findings warrant further investigation and management.",
 

# Adding Routing Agent with Interactive Chat Loop

In [15]:
# ================================
# AGENT 5 — GENERAL QUERY (FALLBACK)
# ================================
# Handles any pregnancy-related question that does NOT fit the four
# specialised agents (symptoms, report, week, daily care).
# Examples: "Can I fly in third trimester?", "Is caffeine safe?"
#
# Returns: { "answer": "...", "disclaimer": "..." }

def general_query(user_input, user_profile):
    """Answer a general pregnancy question, personalised to the patient."""

    logger.info("general_query: called — input='%s'", user_input[:80])

    profile_summary = (
        f"Patient: {user_profile['name']}, Age: {user_profile['age']}, "
        f"Week: {user_profile['pregnancy_week']}, "
        f"Pregnancy type: {user_profile['pregnancy_type']} "
        f"({user_profile['num_fetuses']} fetus(es)), "
        f"Medical history: {', '.join(user_profile['medical_history']) or 'None'}, "
        f"Pregnancy history: {user_profile['pregnancy_history']}"
    )

    output_format = """
{
  "answer": "...",
  "disclaimer": "..."
}
"""

    system_prompt = f"""
You are a helpful prenatal care AI assistant.

Patient profile:
{profile_summary}

Answer the user's general pregnancy-related question in a clear, supportive, and medically responsible way.
Personalize your answer based on the patient's profile when relevant.
If you are unsure or the question requires clinical judgment, advise consulting a doctor.

ALWAYS return your response as a STRICT JSON object. No other text.

Output format:
""" + output_format

    messages = [
        {"role": "system", "content": [{"type": "text", "text": system_prompt}]},
        {"role": "user", "content": [{"type": "text", "text": user_input}]}
    ]

    result = call_model(messages)
    logger.info("general_query: response generated")
    return result

In [16]:
# ================================
# ROUTING AGENT — INTENT CLASSIFIER + DISPATCHER
# ================================
# Two-step process:
#   1. classify_intent() — asks the model to pick ONE tool from the 5 available
#   2. route_query()     — dispatches to the chosen agent and returns the result
#
# If classification fails or returns an unknown tool name, falls back to general_query.

TOOL_DESCRIPTIONS = """
Available tools:
1. analyze_symptoms  — User describes physical symptoms (e.g. pain, swelling, bleeding, nausea, cramps). Use this when the user reports how they feel physically.
2. week_tracker      — User asks about what to expect at their current/specific pregnancy week, trimester info, or milestones.
3. daily_care        — User asks for a daily care plan, diet advice, exercise plan, hydration, or routine recommendations.
4. explain_report    — User shares or asks about a medical report, lab results, blood test, ultrasound findings, or specific medical values.
5. general_query     — Any other pregnancy-related question that doesn't fit the above (e.g. travel, emotions, baby names, general info).
"""

CLASSIFY_OUTPUT_FORMAT = """
{
  "tool": "analyze_symptoms | week_tracker | daily_care | explain_report | general_query",
  "reason": "brief explanation of why this tool was chosen"
}
"""


def classify_intent(user_input):
    """Use the model to classify user intent into one of the available tools."""

    logger.info("classify_intent: classifying — '%s'", user_input[:60])

    system_prompt = """
You are an intent classifier for a prenatal care assistant.

""" + TOOL_DESCRIPTIONS + """

Given the user's message, determine which ONE tool should handle it.

ALWAYS return STRICT JSON ONLY. No other text.

Output format:
""" + CLASSIFY_OUTPUT_FORMAT

    messages = [
        {"role": "system", "content": [{"type": "text", "text": system_prompt}]},
        {"role": "user", "content": [{"type": "text", "text": user_input}]}
    ]

    result = call_model(messages)
    logger.info("classify_intent: tool='%s', reason='%s'",
                result.get("tool", "?"), result.get("reason", "?"))
    return result


def route_query(user_input, user_profile):
    """
    Smart routing agent:
      1. Classifies user intent via the model
      2. Dispatches to the matching agent
      3. Falls back to general_query on unknown tool names
    """
    logger.info("route_query: received — '%s'", user_input[:80])

    # Step 1 — classify
    intent = classify_intent(user_input)
    tool_name = intent.get("tool", "general_query") if isinstance(intent, dict) else "general_query"
    reason = intent.get("reason", "N/A") if isinstance(intent, dict) else "Classification failed"

    logger.info("route_query: dispatching to '%s' (reason: %s)", tool_name, reason)

    # Step 2 — dispatch
    if tool_name == "analyze_symptoms":
        return {"tool_used": tool_name, "result": analyze_symptoms(user_input, user_profile)}

    elif tool_name == "week_tracker":
        return {"tool_used": tool_name, "result": week_tracker(user_profile)}

    elif tool_name == "daily_care":
        return {"tool_used": tool_name, "result": daily_care(user_profile)}

    elif tool_name == "explain_report":
        # The user message itself is treated as the report text
        return {"tool_used": tool_name, "result": explain_report(user_profile, text=user_input)}

    else:
        # Fallback for unknown tool names or general questions
        logger.info("route_query: using general_query fallback")
        return {"tool_used": "general_query", "result": general_query(user_input, user_profile)}

In [142]:
# ================================
# INTERACTIVE CHAT LOOP (Terminal)
# ================================
# A simple REPL that accepts user questions, routes them through
# the routing agent, and pretty-prints the JSON result.
# Type 'quit' or 'exit' to stop.

import json

def pretty_print(result):
    """Display the routing result with tool name and formatted JSON."""
    tool = result.get("tool_used", "unknown")
    data = result.get("result", {})
    print(f"\n[Tool Used: {tool}]")
    print("-" * 40)
    if isinstance(data, dict):
        print(json.dumps(data, indent=2))
    else:
        print(data)
    print("-" * 40)

print("=" * 50)
print("  PRENATAL CARE ASSISTANT (Chat Mode)")
print("  Type your question below.")
print("  Type 'quit' or 'exit' to stop.")
print("=" * 50)

while True:
    user_input = input("\nYou: ").strip()
    if not user_input:
        continue
    if user_input.lower() in ("quit", "exit"):
        print("Goodbye! Take care. 👋")
        break

    result = route_query(user_input, user_profile)
    pretty_print(result)

  PRENATAL CARE ASSISTANT (Chat Mode)
  Type your question below.
  Type 'quit' or 'exit' to stop.



You:  I have swollen feet and back pain, is this normal?


Both `max_new_tokens` (=1024) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



  ROUTING AGENT
  Input: I have swollen feet and back pain, is this normal?...


Both `max_new_tokens` (=1024) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...


  ROUTING AGENT
  Input: Can I travel by flight during my third trimester?...


Both `max_new_tokens` (=1024) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Routed to: general_query
  Reason: The question is about travel during pregnancy, which is a general pregnancy-related inquiry.


[Tool Used: general_query]
----------------------------------------
{
  "answer": "Traveling by air during your third trimester (week 15) is generally considered safe, but it's crucial to consult with your doctor or midwife. They can assess your specific health conditions (diabetes, BP, chest pain) and provide personalized recommendations based on your individual needs and the airline's policies. Some airlines may have restrictions on when pregnant women can fly, especially later in the third trimester. It's also important to consider the length of the flight and the potential for turbulence. Be sure to bring any necessary medications and medical documentation with you.",
  "disclaimer": "This information is for general guidance only and does not substitute professional medical advice. Always consult with your healthcare provider for personalized advice re


You:  quit


Goodbye! Take care.


In [ ]:
# ================================
# QUICK TEST — Route sample queries
# ================================
# These show how different inputs get routed to different tools automatically.

test_queries = [
    "I have swollen feet and back pain, is this normal?",
    "What should I expect at week 24?",
    "Give me a diet and exercise plan for today",
    "Hemoglobin: 9.5 g/dL, BP: 140/90 mmHg — what does this mean?",
    "Can I travel by flight during my third trimester?",
]

for q in test_queries:
    result = route_query(q, user_profile)
    pretty_print(result)
    print("\n" + "=" * 60 + "\n")


# Gradio Live - Prenatal Care Assisstant

In [22]:
!pip install --quiet gradio

In [17]:
# ============================================================
# GRADIO UI — LIVE PRENATAL CARE ASSISTANT (Production-Ready)
# ============================================================

import gradio as gr
import json

# ── Custom CSS — clean spacing, card-style results, light/dark aware ──
CUSTOM_CSS = """
/* ---------- Global ---------- */
.gradio-container { max-width: 960px !important; margin: auto; }
.gr-button { border-radius: 10px !important; font-weight: 600 !important; }

/* ---------- Header banner ---------- */
.app-header { text-align: center; padding: 24px 16px 8px; }
.app-header h1 { font-size: 2rem; margin-bottom: 4px; }
.app-header p  { opacity: 0.75; font-size: 0.95rem; }

/* ---------- Disclaimer ---------- */
.disclaimer-box {
    border-left: 4px solid #e8912d;
    border-radius: 8px;
    padding: 12px 16px;
    margin: 8px 0 16px;
    font-size: 0.9rem;
    line-height: 1.5;
}
/* light theme */
.disclaimer-box { background: #fff8ee; color: #7a5a1f; }
/* dark theme override */
.dark .disclaimer-box { background: #3b2e18; color: #f0d8a8; }

/* ---------- Result cards ---------- */
.result-card {
    border-radius: 12px;
    padding: 20px 24px;
    margin-top: 12px;
    line-height: 1.65;
    font-size: 0.95rem;
    min-height: 120px;
    overflow-y: auto;
}
.result-card { background: #f8f9fc; border: 1px solid #e2e5ec; }
.dark .result-card { background: #1e1e2e; border: 1px solid #3a3a4a; }

.result-card h3 { margin: 14px 0 6px; font-size: 1.05rem; }
.result-card ul { padding-left: 20px; margin: 4px 0; }
.result-card li { margin-bottom: 3px; }

/* Risk badges */
.badge { display: inline-block; padding: 3px 12px; border-radius: 20px; font-weight: 700; font-size: 0.85rem; margin-left: 4px; }
.badge-normal  { background: #d4edda; color: #155724; }
.badge-caution { background: #fff3cd; color: #856404; }
.badge-urgent  { background: #f8d7da; color: #721c24; }
.dark .badge-normal  { background: #1e4620; color: #8fd694; }
.dark .badge-caution { background: #4a3c10; color: #f5d76e; }
.dark .badge-urgent  { background: #4a1c1f; color: #f5a9a9; }

/* ---------- Tab styling ---------- */
.tab-nav button { font-size: 0.95rem !important; padding: 10px 18px !important; }

/* ---------- Profile summary ---------- */
.profile-summary { font-family: monospace; line-height: 1.7; }
"""

CUSTOM_THEME = gr.themes.Soft(primary_hue="pink", secondary_hue="purple")


# ============================================================
# FORMATTING HELPERS — convert raw JSON dicts to readable HTML
# ============================================================

def _risk_badge(level):
    """Return an HTML badge for a risk level string."""
    lvl = (level or "").strip().lower()
    if lvl == "normal":
        return '<span class="badge badge-normal">Normal</span>'
    elif lvl == "caution":
        return '<span class="badge badge-caution">Caution</span>'
    elif lvl == "urgent":
        return '<span class="badge badge-urgent">Urgent</span>'
    return f'<span class="badge">{level}</span>'


def _fmt_error(result):
    """If result contains an error key, return a warning string or None."""
    if isinstance(result, dict) and "error" in result:
        return f"⚠️ **Error:** {result['error']}"
    return None


def fmt_symptoms(result):
    """Format symptom analysis result into readable HTML."""
    err = _fmt_error(result)
    if err:
        return err
    if not isinstance(result, dict):
        return str(result)

    risk = result.get("risk_level", "Unknown")
    return f"""
<div class="result-card">
<h3>Risk Assessment {_risk_badge(risk)}</h3>
<p><strong>Reasoning:</strong> {result.get('reasoning', '—')}</p>
<p><strong>Advice:</strong> {result.get('advice', '—')}</p>
<p><strong>Personalised Note:</strong> {result.get('personalized_note', '—')}</p>
<p><strong>Doctor Visit Required:</strong> {'Yes' if result.get('doctor_required') else 'No'}</p>
</div>"""


def fmt_report(result):
    """Format report explanation into readable HTML."""
    err = _fmt_error(result)
    if err:
        return err
    if not isinstance(result, dict):
        return str(result)

    findings = result.get("findings", [])
    rows = ""
    for f in findings:
        rows += f"""<tr>
            <td><strong>{f.get('name','—')}</strong></td>
            <td>{f.get('value','—')}</td>
            <td>{f.get('meaning','—')}</td>
            <td>{_risk_badge(f.get('risk',''))}</td>
        </tr>"""

    table = f"""
    <table style="width:100%; border-collapse:collapse; margin:8px 0;">
        <thead><tr style="border-bottom:2px solid currentColor; opacity:0.7; text-align:left;">
            <th style="padding:6px 8px;">Test</th>
            <th style="padding:6px 8px;">Value</th>
            <th style="padding:6px 8px;">Meaning</th>
            <th style="padding:6px 8px;">Risk</th>
        </tr></thead>
        <tbody>{rows}</tbody>
    </table>""" if rows else ""

    return f"""
<div class="result-card">
<h3>📋 Report Findings</h3>
{table}
<p><strong>Summary:</strong> {result.get('summary', '—')}</p>
<p><strong>Personalised Note:</strong> {result.get('personalized_note', '—')}</p>
</div>"""


def fmt_week(result):
    """Format week tracker result into readable HTML."""
    err = _fmt_error(result)
    if err:
        return err
    if not isinstance(result, dict):
        return str(result)

    tips = "".join(f"<li>{t}</li>" for t in result.get("tips", []))
    symptoms = "".join(f"<li>{s}</li>" for s in result.get("symptoms", []))
    reminders = "".join(f"<li>{r}</li>" for r in result.get("doctor_reminders", []))

    return f"""
<div class="result-card">
<h3>📅 Trimester: {result.get('trimester', '—')}</h3>
<h3>💡 Tips</h3><ul>{tips or '<li>—</li>'}</ul>
<h3>🤒 Expected Symptoms</h3><ul>{symptoms or '<li>—</li>'}</ul>
<h3>🩺 Doctor Reminders</h3><ul>{reminders or '<li>—</li>'}</ul>
<p><strong>Personalised Note:</strong> {result.get('personalized_note', '—')}</p>
</div>"""


def fmt_care(result):
    """Format daily care plan into readable HTML."""
    err = _fmt_error(result)
    if err:
        return err
    if not isinstance(result, dict):
        return str(result)

    diet = "".join(f"<li>{d}</li>" for d in result.get("diet", []))
    avoid = "".join(f"<li>{a}</li>" for a in result.get("foods_to_avoid", []))
    exercise = "".join(f"<li>{e}</li>" for e in result.get("exercise", []))

    return f"""
<div class="result-card">
<h3>🥗 Diet</h3><ul>{diet or '<li>—</li>'}</ul>
<h3>🚫 Foods to Avoid</h3><ul>{avoid or '<li>—</li>'}</ul>
<h3>💧 Hydration</h3><p>{result.get('hydration', '—')}</p>
<h3>🏃‍♀️ Exercise</h3><ul>{exercise or '<li>—</li>'}</ul>
<h3>😴 Rest</h3><p>{result.get('rest_recommendation', '—')}</p>
<h3>📝 Special Notes</h3><p>{result.get('special_notes', '—')}</p>
</div>"""


def fmt_general(result):
    """Format general query response into readable HTML."""
    err = _fmt_error(result)
    if err:
        return err
    if not isinstance(result, dict):
        return str(result)

    return f"""
<div class="result-card">
<p>{result.get('answer', '—')}</p>
<p style="opacity:0.65; font-size:0.85rem; margin-top:12px;">
⚕️ <em>{result.get('disclaimer', '')}</em></p>
</div>"""


def fmt_chat_result(routed):
    """Format a routing-agent result for the chatbot."""
    tool = routed.get("tool_used", "unknown")
    result = routed.get("result", {})

    # Pick the right formatter
    formatters = {
        "analyze_symptoms": fmt_symptoms,
        "explain_report": fmt_report,
        "week_tracker": fmt_week,
        "daily_care": fmt_care,
        "general_query": fmt_general,
    }
    formatter = formatters.get(tool, fmt_general)
    body = formatter(result)

    tool_label = tool.replace("_", " ").title()
    return f"🔧 **Agent:** {tool_label}\n\n{body}"


# ============================================================
# TAB HANDLERS
# ============================================================

def save_profile(name, age, weight, height, week, preg_type, num_fetuses,
                 med_history, allergies, blood_group, first_pregnancy):
    pregnancy_history = (
        "First Pregnancy (has no previous pregnancy experience)"
        if first_pregnancy == "Yes"
        else "Not First Pregnancy (has previous pregnancy experience)"
    )
    profile = {
        "name": name.strip() or "Patient",
        "age": int(age),
        "weight_kg": float(weight),
        "height_cm": float(height),
        "pregnancy_week": int(week),
        "pregnancy_type": preg_type.lower(),
        "num_fetuses": int(num_fetuses) if preg_type.lower() == "multiple" else 1,
        "medical_history": [h.strip() for h in med_history.split(",") if h.strip()] if med_history.strip().lower() not in ("none", "") else [],
        "allergies": [a.strip() for a in allergies.split(",") if a.strip()] if allergies.strip().lower() not in ("none", "") else [],
        "blood_group": blood_group.strip() or "Unknown",
        "pregnancy_history": pregnancy_history,
    }
    logger.info("Gradio save_profile: saved for '%s' (week %d)", profile["name"], profile["pregnancy_week"])

    lines = [f"**{k.replace('_',' ').title()}:** {v}" for k, v in profile.items()]
    md = "✅ **Profile saved successfully!**\n\n" + "\n\n".join(lines)
    return profile, md


def symptom_handler(symptoms_text, profile):
    if not profile:
        return "⚠️ Please save your profile first in the **Profile Setup** tab."
    logger.info("Gradio symptom_handler: '%s'", symptoms_text[:60])
    result = analyze_symptoms(symptoms_text, profile)
    return fmt_symptoms(result)


def report_handler(report_text, report_image, profile):
    if not profile:
        return "⚠️ Please save your profile first in the **Profile Setup** tab."
    img = None
    if report_image is not None:
        from PIL import Image
        img = Image.open(report_image) if isinstance(report_image, str) else report_image
    logger.info("Gradio report_handler: text=%s, image=%s", bool(report_text), bool(img))
    result = explain_report(profile, text=report_text or None, image=img)
    return fmt_report(result)


def week_handler(profile):
    if not profile:
        return "⚠️ Please save your profile first in the **Profile Setup** tab."
    logger.info("Gradio week_handler: week %d", profile.get("pregnancy_week", 0))
    result = week_tracker(profile)
    return fmt_week(result)


def care_handler(profile):
    if not profile:
        return "⚠️ Please save your profile first in the **Profile Setup** tab."
    logger.info("Gradio care_handler: generating plan")
    result = daily_care(profile)
    return fmt_care(result)


def chat_handler(user_msg, history, profile):
    if not profile:
        history = history or []
        history.append({"role": "assistant", "content": "⚠️ Please save your profile first in the **Profile Setup** tab."})
        return history, ""
    logger.info("Gradio chat_handler: '%s'", user_msg[:60])
    routed = route_query(user_msg, profile)
    reply = fmt_chat_result(routed)
    history = history or []
    history.append({"role": "user", "content": user_msg})
    history.append({"role": "assistant", "content": reply})
    logger.info("Gradio chat_handler: routed to '%s'", routed.get("tool_used"))
    return history, ""


# ============================================================
# BUILD THE GRADIO APP
# ============================================================
with gr.Blocks(title="MedGemma Prenatal Care Assistant") as demo:

    profile_state = gr.State(value={})

    # ── Header ────────────────────────────────────────────────
    gr.HTML("""
    <div class="app-header">
        <h1>🤰 MedGemma Prenatal Care Assistant</h1>
        <p>Personalised pregnancy guidance powered by <strong>Google MedGemma 4B</strong></p>
    </div>
    <div class="disclaimer-box">
        ⚠️ <strong>Disclaimer:</strong> This tool is for <em>informational purposes only</em> and
        does <strong>not</strong> replace professional medical advice. Always consult your
        healthcare provider for clinical decisions.
    </div>
    """)

    # ── Tab 1: Profile ────────────────────────────────────────
    with gr.Tab("👤 Profile Setup"):
        gr.Markdown("Fill in your details once — they're used by **every agent** for personalised recommendations.")
        with gr.Row(equal_height=True):
            with gr.Column(scale=1):
                inp_name   = gr.Textbox(label="Name", placeholder="e.g. Priya")
                inp_age    = gr.Number(label="Age", value=28, minimum=14, maximum=60)
                inp_weight = gr.Number(label="Weight (kg)", value=60)
                inp_height = gr.Number(label="Height (cm)", value=160)
                inp_week   = gr.Slider(label="Pregnancy Week", minimum=1, maximum=42, step=1, value=12)
            with gr.Column(scale=1):
                inp_type    = gr.Radio(["Single", "Multiple"], label="Pregnancy Type", value="Single")
                inp_fetuses = gr.Number(label="Number of Fetuses (if multiple)", value=1, minimum=1, maximum=5)
                inp_history = gr.Textbox(label="Medical History", placeholder="e.g. diabetes, hypertension  (or leave blank)")
                inp_allergy = gr.Textbox(label="Allergies", placeholder="e.g. peanuts, penicillin  (or leave blank)")
                inp_blood   = gr.Textbox(label="Blood Group", placeholder="e.g. A+, O-")
                inp_first   = gr.Radio(["Yes", "No"], label="First Pregnancy?", value="Yes")

        btn_save = gr.Button("💾  Save Profile", variant="primary", size="lg")
        profile_out = gr.Markdown(elem_classes=["profile-summary"])

        btn_save.click(
            save_profile,
            inputs=[inp_name, inp_age, inp_weight, inp_height, inp_week,
                    inp_type, inp_fetuses, inp_history, inp_allergy, inp_blood, inp_first],
            outputs=[profile_state, profile_out],
        )

    # ── Tab 2: Symptoms ───────────────────────────────────────
    with gr.Tab("🩺 Symptom Analyzer"):
        gr.Markdown("Describe how you're feeling and get a **personalised risk assessment**.")
        sym_input = gr.Textbox(label="Your Symptoms", lines=3,
                               placeholder="e.g. I have swollen feet and lower back pain since morning…")
        sym_btn = gr.Button("Analyse Symptoms", variant="primary")
        sym_out = gr.HTML(label="Analysis")
        sym_btn.click(symptom_handler, inputs=[sym_input, profile_state], outputs=sym_out)

    # ── Tab 3: Report ─────────────────────────────────────────
    with gr.Tab("📋 Report Explainer"):
        gr.Markdown("Paste your **lab report text** or upload an **image** — the AI will explain each finding.")
        with gr.Row():
            rep_text  = gr.Textbox(label="Report Text", lines=6,
                                   placeholder="e.g.\nHemoglobin: 9.5 g/dL\nBP: 140/90 mmHg")
            rep_image = gr.Image(label="Upload Report Image (optional)", type="pil")
        rep_btn = gr.Button("Explain Report", variant="primary")
        rep_out = gr.HTML(label="Report Analysis")
        rep_btn.click(report_handler, inputs=[rep_text, rep_image, profile_state], outputs=rep_out)

    # ── Tab 4: Week Tracker ───────────────────────────────────
    with gr.Tab("📅 Week Tracker"):
        gr.Markdown("Get **tips, expected symptoms, and doctor reminders** for your current pregnancy week.")
        wk_btn = gr.Button("Get Week Info", variant="primary", size="lg")
        wk_out = gr.HTML(label="Week Tracker")
        wk_btn.click(week_handler, inputs=[profile_state], outputs=wk_out)

    # ── Tab 5: Daily Care ─────────────────────────────────────
    with gr.Tab("🥗 Daily Care Plan"):
        gr.Markdown("Generate a **personalised daily plan** covering diet, exercise, hydration, and rest.")
        dc_btn = gr.Button("Generate Daily Care Plan", variant="primary", size="lg")
        dc_out = gr.HTML(label="Daily Care Plan")
        dc_btn.click(care_handler, inputs=[profile_state], outputs=dc_out)

    # ── Tab 6: Smart Chat ─────────────────────────────────────
    with gr.Tab("💬 Smart Chat"):
        gr.Markdown("Ask **anything** — the routing agent picks the right tool and formats the answer for you.")
        chatbot = gr.Chatbot(label="Prenatal Assistant", height=420, type="messages", allow_tags=True)
        with gr.Row():
            chat_input = gr.Textbox(label="Your message", placeholder="Type your question…",
                                    lines=1, scale=4, show_label=False)
            chat_btn = gr.Button("Send ➤", variant="primary", scale=1)
        chat_btn.click(chat_handler, inputs=[chat_input, chatbot, profile_state], outputs=[chatbot, chat_input])
        chat_input.submit(chat_handler, inputs=[chat_input, chatbot, profile_state], outputs=[chatbot, chat_input])

    # ── Footer ────────────────────────────────────────────────
    gr.HTML("""
    <div style="text-align:center; opacity:0.5; font-size:0.8rem; padding:16px 0 8px;">
        Built with MedGemma + Gradio &nbsp;·&nbsp; For educational purposes only
    </div>
    """)

logger.info("Gradio app built — 6 tabs ready")

[12:25:13] INFO | Gradio app built — 6 tabs ready


In [ ]:
# Launch the Gradio app
# - On Kaggle/Colab: share=True gives you a public URL (valid ~72 hrs)
# - Locally: opens at http://127.0.0.1:7860

demo.launch(share=True, debug=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://307c4895b22f48406c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[12:26:41] INFO | Gradio save_profile: saved for 'Priya' (week 12)
[12:27:19] INFO | Gradio symptom_handler: 'i have mild headache
'
[12:27:19] INFO | analyze_symptoms: called — input='i have mild headache
'
[12:27:19] INFO | call_model: attempt 1/3
Passing `generation_config` together with generation-related arguments=({'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=1024) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
[12:27:29] INFO | Gradio report_handler: text=False, image=True
[12:27:29] INFO | explain_report: called — text=False, image=True
[12:27:29] INFO | call_model: attempt 1/3
Both `max_new_tokens` (=1024) and `max_length`(=20) seem to have been set. `max